In [95]:
# Play around CCA using sklearn
import numpy as np
import pandas as pd
import nibabel as nib
import nilearn as nil
import matplotlib.pyplot as plt
import seaborn as sns 
import csv
import sklearn
from sklearn.cross_decomposition import CCA

In [96]:
sklearn.__version__

'0.22'

In [97]:
# get behav data from Marco
Sub = np.loadtxt('/data/backed_up/shared/mpipoly_HCPtest/batch_data/subjects.txt', dtype=str)
Task1=['WM', 'SOCIAL', 'RELATIONAL', 'GAMBLING', 'EMOTION', 'MOTOR', 'LANGUAGE','rfMRI','task']
data_dir = '/data/backed_up/shared/mpipoly_HCPtest/batch_data/'

csv2 = pd.read_csv('/data/backed_up/shared/mpipoly_HCPtest/batch_data/PYTHON_sTATS/unrestricted_mpipoly_11_19_2019_12_1_22.csv') 
df2=csv2[csv2['Subject'].isin(list(Sub))] 

newDF2=df2[['Emotion_Task_Acc','Emotion_Task_Median_RT','Language_Task_Acc','Language_Task_Median_RT','Gambling_Task_Median_RT_Larger','Gambling_Task_Median_RT_Smaller','Relational_Task_Acc','Relational_Task_Median_RT','WM_Task_Acc','WM_Task_Median_RT','Social_Task_Median_RT_TOM','Social_Task_Median_RT_Random','Social_Task_Median_RT_Unsure']].copy()

In [98]:
#get FC matrices
task_keys={}
#Glasser_Variate_X=np.array([])
#Gordon_Variate_X=np.array([])
final={}
for t in Task1:
    task_keys[t]={}
    ls_Glass=list()
    ls_Gord=list()
    final[t]={}
    for s in Sub:
        task_keys[t][s]={}
        task_keys[t][s]['Thal_X_Gord'] = (data_dir+'PYTHON_sTATS'+'/'+'{0}'+'/'+'{1}'+'/'+'{0}'+'_'+'{1}'+'_'+'Thal_X_Gord'+'.npy').format(s,t)
        #task_keys[s][t]['Thal_X_Gord'] = task_level[s]['rfMRI'].update({'Thal_X_Gord': task_level['GordParc'].get_fdata()})
        task_keys[t][s]['Thal_X_Gord'] = np.load(task_keys[t][s]['Thal_X_Gord'])
        task_keys[t][s]['Thal_X_Glass'] = (data_dir+'PYTHON_sTATS'+'/'+'{0}'+'/'+'{1}'+'/'+'{0}'+'_'+'{1}'+'_'+'Thal_X_Glass'+'.npy').format(s,t)
        task_keys[t][s]['Thal_X_Glass'] = np.load(task_keys[t][s]['Thal_X_Glass'])
        task_keys[t][s]['Thal_X_Gord'] = task_keys[t][s]['Thal_X_Gord'].flatten()
        task_keys[t][s]['Thal_X_Glass'] = task_keys[t][s]['Thal_X_Glass'].flatten()
        ls_Glass.append(np.array([task_keys[t][s]['Thal_X_Glass']]))
        ls_Gord.append(np.array(task_keys[t][s]['Thal_X_Gord']))
        #Glasser_Variate_X=np.vstack((Glasser_Variate_X,Tmp_Glass))
        #Glasser_Variate_X=np.append(Glasser_Variate_X, task_keys[t][s]['Thal_X_Glass'])
        #Gordon_Variate_X=np.append(Gordon_Variate_X, task_keys[t][s]['Thal_X_Gord'])
    final[t]={'Gordon_Variate_X': np.vstack(ls_Gord)}
    final[t].update({'Glasser_Variate_X': np.vstack(ls_Gord)})
    #print(final['WM']['Glasser_Variate_X'])
    #/data/backed_up/shared/mpipoly_HCPtest/batch_data/PYTHON_sTATS/task/CCA_Analyses
    np.save(f'/data/backed_up/shared/mpipoly_HCPtest/batch_data/PYTHON_sTATS/task/CCA_Analyses/{t}_ALL_cca_Thal_X_Gord', final[t]['Gordon_Variate_X'])
    np.save(f'/data/backed_up/shared/mpipoly_HCPtest/batch_data/PYTHON_sTATS/task/CCA_Analyses/{t}_ALL_cca_Thal_X_Glass', final[t]['Glasser_Variate_X'])
    
print(final['WM']['Glasser_Variate_X']) # Check Stacking
print(len(final['WM']['Glasser_Variate_X'])) # Check 100 flattened arrays are present
print(final['WM'].keys())

[[ 0.0261526   0.05646732  0.02090673 ... -0.01015351 -0.06936608
   0.1371647 ]
 [-0.00525365  0.07348273  0.01819496 ...  0.15467393 -0.05287683
   0.0529021 ]
 [ 0.08448181 -0.00367428 -0.03634613 ... -0.027576    0.04881956
   0.02245879]
 ...
 [-0.01173314  0.12081812 -0.0515199  ... -0.00310758  0.04449047
  -0.06093103]
 [-0.0188491   0.08961105 -0.00266385 ... -0.00866891 -0.04057553
   0.05887987]
 [ 0.01949048  0.00064084 -0.09660682 ... -0.08229227 -0.02524862
  -0.01831261]]
100
dict_keys(['Gordon_Variate_X', 'Glasser_Variate_X'])


In [99]:
X = final['WM']['Glasser_Variate_X']
print(X.shape)
print(X)

(100, 844488)
[[ 0.0261526   0.05646732  0.02090673 ... -0.01015351 -0.06936608
   0.1371647 ]
 [-0.00525365  0.07348273  0.01819496 ...  0.15467393 -0.05287683
   0.0529021 ]
 [ 0.08448181 -0.00367428 -0.03634613 ... -0.027576    0.04881956
   0.02245879]
 ...
 [-0.01173314  0.12081812 -0.0515199  ... -0.00310758  0.04449047
  -0.06093103]
 [-0.0188491   0.08961105 -0.00266385 ... -0.00866891 -0.04057553
   0.05887987]
 [ 0.01949048  0.00064084 -0.09660682 ... -0.08229227 -0.02524862
  -0.01831261]]


In [239]:
Acc=newDF2['WM_Task_Acc']
RT=newDF2['WM_Task_Median_RT']
Y = np.squeeze(np.dstack((Acc,RT)))


In [216]:
#try CCA, and print correlation. These correlations are called the Canonical correlations
cca1 = CCA(n_components=2)
cca1.fit(X, Y)
X_c, Y_c = cca1.fit_transform(X,Y)

print(np.corrcoef(cca1.x_scores_.T, cca1.y_scores_.T))


[[ 1.00000000e+00  1.45738866e-17  1.00000000e+00 -4.45190550e-17]
 [ 1.45738866e-17  1.00000000e+00 -1.22807133e-16  1.00000000e+00]
 [ 1.00000000e+00 -1.22807133e-16  1.00000000e+00 -4.77348547e-17]
 [-4.45190550e-17  1.00000000e+00 -4.77348547e-17  1.00000000e+00]]


In [217]:
# weights are the A and B in AX = BY # loadings and rotations are the PCA loadings which we will not use
print(cca1.x_loadings_.shape)
print(cca1.x_weights_.shape)
print(cca1.x_rotations_.shape)

(844488, 2)
(844488, 2)
(844488, 2)


In [218]:
cca1.x_scores_.shape
X.shape

(100, 844488)

In [219]:
# this is the steve smith approach where we correlate scores with original FC
r_vals = np.zeros(844488)

for i in np.arange(844488):
    r_vals[i]=np.corrcoef(cca1.x_scores_[:,0], X[:,i])[0,1]
    
    

In [220]:
print(np.mean(r_vals))
print(np.std(r_vals))
print(np.max(r_vals))


-2.674299029493056e-05
0.1030936836368332
0.4865108385465558


In [237]:
#Let us compare the wieghts
XWM = final['WM']['Glasser_Variate_X']
AccWM=newDF2['WM_Task_Acc']
RTWM=newDF2['WM_Task_Median_RT']
Y = np.squeeze(np.dstack((Acc,RT)))
cca1 = CCA(n_components=2)
cca1.fit(XWM, Y)

XL = final['SOCIAL']['Glasser_Variate_X']
S1=newDF2['Social_Task_Median_RT_Unsure']
S2=newDF2['Social_Task_Median_RT_TOM']
YS = np.squeeze(np.dstack((S1,S2)))
YS[np.isnan(YS)]=0
cca2 = CCA(n_components=2)
cca2.fit(XL, YS)




CCA(copy=True, max_iter=500, n_components=2, scale=True, tol=1e-06)

In [238]:
np.corrcoef(cca1.x_weights_.T, cca2.x_weights_.T) # are they correlated at all?

array([[ 1.00000000e+00, -1.38302098e-04, -4.12809404e-04,
         7.52505226e-03],
       [-1.38302098e-04,  1.00000000e+00,  3.99152540e-03,
         1.13462001e-03],
       [-4.12809404e-04,  3.99152540e-03,  1.00000000e+00,
        -2.89557062e-04],
       [ 7.52505226e-03,  1.13462001e-03, -2.89557062e-04,
         1.00000000e+00]])

In [240]:
from statsmodels.multivariate.cancorr import CanCorr

In [253]:
ts=CanCorr(XWM, Y)